# Information retrieval and Search
- We have a lot of documents, we hope to extract their significant characteristics compared to others.

## Build Data
Each doc has been tokenzied to a word list

In [43]:
import json
import pandas as pd

with open("data/reuters.json") as fin:
    documents = json.load(fin)

print("Number of documents: %d" % len(documents))
print(documents[0].keys()) # field: title, content
print(documents[0]['content'][:100])

Number of documents: 5669
dict_keys(['title', 'content'])
['Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.', 'They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.', 'But', 'some', 'exporters', 'said', 'that', 'while', 'the', 'conflict', 'would', 'hurt', 'them', 'in', 'the', 'long', '-', 'run', ',', 'in', 'the', 'short', '-', 'term', 'Tokyo', "'", 's', 'loss']


In [69]:
doc_df = pd.DataFrame(documents)
doc_df

,title,content
0,ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPA...,"[Mounting, trade, friction, between, the, U, ...."
1,CHINA DAILY SAYS VERMIN EAT 7 - 12 PCT GRAIN S...,"[survey, of, 19, provinces, and, seven, cities..."
2,JAPAN TO REVISE LONG - TERM ENERGY DEMAND DOWN...,"[The, Ministry, of, International, Trade, and,..."
3,THAI TRADE DEFICIT WIDENS IN FIRST QUARTER,"[Thailand, ', s, trade, deficit, widened, to, ..."
4,INDONESIA SEES CPO PRICE RISING SHARPLY,"[Indonesia, expects, crude, palm, oil, (, CPO,..."
...,...,...
5664,FED SETS TWO BILLION DLR CUSTOMER REPURCHASE,"[The, Federal, Reserve, entered, the, U, ., S,..."
5665,SENATE SEEKS U . S . PROBE OF CANADIAN CORN LEVY,"[The, Senate, voted, unanimously, to, seek, an..."
5666,U . K . MONEY MARKET SHORTAGE FORECAST REVISED...,"[The, Bank, of, England, said, it, had, revise..."
5667,KNIGHT - RIDDER INC &,"[lt, ;, KRN, >, SETS, QUARTERLY, Qtly, div, 25..."


## Search for a single query term
`search()`: return all docs containing query term

In [25]:
def search(docs, query):
    results = []
    for doc in docs:
        if query in doc['content']:
            results.append(doc['title'])
    return results


query = "computer"

print(len(search(documents, query)))
print(search(documents, query))

54
['VW SAYS 480 MLN MARKS MAXIMUM FOR CURRENCY LOSSES', 'JAPAN WARNS U . S . IT MAY RETALIATE IN TRADE DISPUTE', 'SINGAPORE EXTERNAL TRADE GAINS 8 . 8 PCT IN QUARTER', 'TALKING POINT / IBM &', 'YEUTTER SAYS JAPANESE CURB ALL BUT CERTAIN U . S .', 'COMPUTERLAND TO BE ACQUIRED BY INVESTOR GROUP', 'ENTERTAINMENT MARKETING TOPS CRAZY EDDIE OFFER A', 'INVESTORS MAY TAKE COMPUTERLAND PUBLIC', 'SHULTZ WELCOMES TOKYO ECONOMIC PACKAGE U . S .', 'TOSHIBA , SHARP RESTRAIN LAP - TOP PC EXPORTS TO EC', 'MITSUI BUYS FIVE PCT STAKE IN U . S . CHIP MAKER', 'NEW LEADER COMING TO U . S . SEC IN CHALLENGING ERA', 'NAKASONE HARD - PRESSED TO SOOTHE U . S ANGER ON TRADE', 'HONEYWELL BULL SEES REVENUE GROWTH', "JAPAN ' S CHIP MAKERS ANGERED BY U . S . SANCTION PLANS", 'NAKASONE SOUNDS CONCILIATORY NOTE IN CHIP DISPUTE', 'TOKYO BIDS TO STOP CHIP ROW TURNING INTO TRADE WAR', 'JAPAN HAS LITTLE NEW TO OFFER IN MICROCHIP DISPUTE', 'NAKASONE SOUNDS CONCILIATORY NOTE IN CHIP DISPUTE', 'TOKYO BIDS TO STOP CHIP ROW

## Search for multiple terms (AND)

In [26]:

# match if all query terms appears in the doc
# match if a list appears in a doc
def match(doc_terms, query_terms):
    for term in query_terms:
        if term not in doc_terms:
            return False
    return True


def match1(doc_terms, query_terms):
    return set(query_terms) <= set(doc_terms)


def search(docs, query):
    query_terms = query.split(" ")
    results = []
    for doc in docs:
        if match(doc['content'], query_terms):
            results.append(doc['title'])
    return results

query = "Japan beef"
results = search(documents, query)
print("Number of matched results: %d" % len(results))
print("\n".join(results[0:10]))

Number of matched results: 21
ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT
JAPAN MINISTRY SAYS OPEN FARM TRADE WOULD HIT U . S .
JAPAN ' S LDP URGES MORE IMPORTS OF 12 FARM ITEMS
U . S . URGES JAPAN TO OPEN FARM MARKET FURTHER U . S .
LYNG OPENS JAPAN TALKS ON FARM TRADE BARRIERS U . S .
U . S . URGES JAPAN TO OPEN FARM MARKET FURTHER U . S .
U . S . MEAT INDUSTRY LAUNCHES CAMPAIGN IN JAPAN
GATT CASE AGAINST JAPAN A MODEL FOR U . S . - LYNG
U . S . TAKES TOUGH STAND ON GATT FARM ISSUES
EC AGREES TRADE DEAL WITH ARGENTINA


# TF-IDF
- **tf**: The occurrences of a term in a document. *tf(t, d)*
- **df**: document frequency: Number of docs where a word appreared
- **tf-idf**: term-frequency and inverted document frequency

## Testing tf-idf

In [21]:
def print_tfidf(td, ndoc, ntd):
    import math
    idf = math.log(ndoc/ntd)
    tfidf = td*idf
    print("n(t, d)=%d\t ndoc=%d\t ndoc_term=%d\t idf=%f\t tfidf=%f"%(td, ndoc, ntd, idf, tfidf))

print_tfidf(5, 100, 10)
print_tfidf(10, 100, 20)
print_tfidf(20, 100, 40)
print_tfidf(40, 100, 80)
print_tfidf(50, 100, 100)

print("-"*80)

print_tfidf(5, 100, 5)
print_tfidf(5, 100, 10)
print_tfidf(5, 100, 20)
print_tfidf(5, 100, 40)

print("-"*80)

print_tfidf(5, 100, 5)
print_tfidf(10, 100, 10)
print_tfidf(20, 100, 20)
print_tfidf(40, 100, 40)



n(t, d)=5	 ndoc=100	 ndoc_term=10	 idf=2.302585	 tfidf=11.512925
n(t, d)=10	 ndoc=100	 ndoc_term=20	 idf=1.609438	 tfidf=16.094379
n(t, d)=20	 ndoc=100	 ndoc_term=40	 idf=0.916291	 tfidf=18.325815
n(t, d)=40	 ndoc=100	 ndoc_term=80	 idf=0.223144	 tfidf=8.925742
n(t, d)=50	 ndoc=100	 ndoc_term=100	 idf=0.000000	 tfidf=0.000000
--------------------------------------------------------------------------------
n(t, d)=5	 ndoc=100	 ndoc_term=5	 idf=2.995732	 tfidf=14.978661
n(t, d)=5	 ndoc=100	 ndoc_term=10	 idf=2.302585	 tfidf=11.512925
n(t, d)=5	 ndoc=100	 ndoc_term=20	 idf=1.609438	 tfidf=8.047190
n(t, d)=5	 ndoc=100	 ndoc_term=40	 idf=0.916291	 tfidf=4.581454
--------------------------------------------------------------------------------
n(t, d)=5	 ndoc=100	 ndoc_term=5	 idf=2.995732	 tfidf=14.978661
n(t, d)=10	 ndoc=100	 ndoc_term=10	 idf=2.302585	 tfidf=23.025851
n(t, d)=20	 ndoc=100	 ndoc_term=20	 idf=1.609438	 tfidf=32.188758
n(t, d)=40	 ndoc=100	 ndoc_term=40	 idf=0.916291	 tfidf=3

### Compute tf and df

In [70]:
from collections import Counter

term_freq = Counter()
doc_freq = Counter() # Number of docs where a word appeared

tdf = list()
for doc in documents:
    term_doc_freq = Counter()
    doc_terms = set() # to store all terms appearing in the doc
    for w in doc['content']:
        if len(w) > 1:
            term_freq[w] += 1
            term_doc_freq[w] += 1
            doc_terms.add(w)
        
    for w in doc_terms:
        doc_freq[w] += 1
    tdf.append(term_doc_freq)
doc_df['tdf'] = tdf
doc_df

,title,content,tdf
0,ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPA...,"[Mounting, trade, friction, between, the, U, ....","{'Mounting': 1, 'trade': 13, 'friction': 1, 'b..."
1,CHINA DAILY SAYS VERMIN EAT 7 - 12 PCT GRAIN S...,"[survey, of, 19, provinces, and, seven, cities...","{'survey': 1, 'of': 5, '19': 1, 'provinces': 1..."
2,JAPAN TO REVISE LONG - TERM ENERGY DEMAND DOWN...,"[The, Ministry, of, International, Trade, and,...","{'The': 2, 'Ministry': 1, 'of': 8, 'Internatio..."
3,THAI TRADE DEFICIT WIDENS IN FIRST QUARTER,"[Thailand, ', s, trade, deficit, widened, to, ...","{'Thailand': 2, 'trade': 1, 'deficit': 1, 'wid..."
4,INDONESIA SEES CPO PRICE RISING SHARPLY,"[Indonesia, expects, crude, palm, oil, (, CPO,...","{'Indonesia': 3, 'expects': 1, 'crude': 1, 'pa..."
...,...,...,...
5664,FED SETS TWO BILLION DLR CUSTOMER REPURCHASE,"[The, Federal, Reserve, entered, the, U, ., S,...","{'The': 1, 'Federal': 2, 'Reserve': 1, 'entere..."
5665,SENATE SEEKS U . S . PROBE OF CANADIAN CORN LEVY,"[The, Senate, voted, unanimously, to, seek, an...","{'The': 2, 'Senate': 2, 'voted': 1, 'unanimous..."
5666,U . K . MONEY MARKET SHORTAGE FORECAST REVISED...,"[The, Bank, of, England, said, it, had, revise...","{'The': 1, 'Bank': 1, 'of': 3, 'England': 1, '..."
5667,KNIGHT - RIDDER INC &,"[lt, ;, KRN, >, SETS, QUARTERLY, Qtly, div, 25...","{'lt': 1, 'KRN': 1, 'SETS': 1, 'QUARTERLY': 1,..."


### Compute idf

In [72]:
import math
num_docs = len(documents)
tfidfs = list()
for tdf in doc_df['tdf']:
    tfidf = Counter()
    for w, f in tdf.items():
        tfidf[w] = f * math.log2(num_docs/doc_freq[w])
    tfidfs.append(tfidf)
doc_df['tfidf'] = tfidfs

In [73]:
for title, tfidf in zip(doc_df['title'][:20], doc_df['tfidf'][:20]):
    print(title.lower())
    for k, v in tfidf.most_common(5):
        print("\t%s(%.4f)"%(k, v))

asian exporters fear damage from u . s .- japan rift
	Japan(40.0751)
	trade(35.1953)
	electronics(31.3001)
	tariffs(29.1976)
	businessmen(28.4453)
china daily says vermin eat 7 - 12 pct grain stocks a
	preservation(21.7678)
	waste(18.0189)
	China(15.8372)
	storage(13.5929)
	vermin(12.4689)
japan to revise long - term energy demand downwards
	energy(33.8159)
	MITI(31.0738)
	electric(16.7628)
	demand(15.6265)
	kilolitres(12.4689)
thai trade deficit widens in first quarter
	baht(30.4409)
	Thailand(13.5929)
	billion(13.4837)
	pct(13.3317)
	Janunary(12.4689)
indonesia sees cpo price rising sharply
	Harahap(37.4066)
	CPO(31.4066)
	palm(27.1858)
	Indonesia(18.7783)
	Indonesian(14.1531)
australian foreign ship ban ends but nsw ports hit
	NSW(41.8755)
	ports(28.1705)
	disrupted(17.3230)
	disruption(17.3230)
	shipping(16.8702)
indonesian commodity exchange may expand
	rubber(48.1874)
	exchange(38.7578)
	Nainggolan(37.4066)
	coffee(26.9603)
	trading(22.9718)
sri lanka gets usda approval for wheat

### (Testing) tfidf for whole doc

In [30]:
import math

tfidf = Counter() # tfidf of each word
num_docs = len(documents)
for w in term_freq:
    tfidf[w] = term_freq[w] * math.log2(num_docs/doc_freq[w])

print("Most frequent terms:")
print(term_freq.most_common(50))

print("Most frequent document terms:")
print(doc_freq.most_common(50))

print("Terms with the highest TFIDF:")
print(tfidf.most_common(50))

Most frequent terms:
[('.', 65574), ('the', 47222), (',', 46891), ('to', 26853), ('of', 25821), ('in', 21209), ('and', 18750), ('said', 18485), ('a', 17638), ('for', 9145), ('mln', 9128), ('-', 9003), ('The', 8825), ("'", 8457), ('pct', 7529), ('s', 7246), ('on', 6853), ('from', 6145), ('that', 6118), ('is', 5835), ('"', 5775), ('dlrs', 5654), ('by', 5533), ('1', 5510), ('at', 5409), ('it', 5379), ('was', 4969), ('be', 4906), ('year', 4748), ('with', 4704), ('U', 4604), ('000', 4450), ('S', 4418), ('its', 4337), ('billion', 4320), ('vs', 4315), ('will', 4115), ('would', 3906), ('2', 3738), ('as', 3433), (';', 3400), ('lt', 3349), ('/', 3327), ('has', 3324), ('not', 3285), ('an', 3273), ('he', 3159), (',"', 2990), ('which', 2873), ('3', 2861)]
Most frequent document terms:
[('.', 5518), (',', 5260), ('of', 4774), ('the', 4675), ('said', 4608), ('to', 4505), ('and', 4409), ('in', 4328), ('a', 4186), ('The', 3645), ('for', 3481), ('-', 3132), ("'", 3066), ('s', 2929), ('on', 2869), ('from

In [68]:
from nltk.corpus import stopwords
stopword_list = stopwords.words('english')

term_freq = Counter()
doc_freq = Counter()

for doc in documents:
    doc_terms = set()
    for w in doc['content']:
        if not w.isalpha():
            continue
        w = w.lower()
        if w not in stopword_list:
            term_freq[w] += 1
            doc_terms.add(w)
    for w in doc_terms:
        doc_freq[w] += 1


tfidf = Counter()
num_docs = len(documents)
for w in term_freq:
    if w not in stopword_list and len(w) > 2:
        tfidf[w] = term_freq[w] * math.log2(num_docs/doc_freq[w])

print("Most frequent terms:")
print(term_freq.most_common(50))

print("Most frequent document terms:")
print(doc_freq.most_common(50))

print("Terms with the highest TFIDF:")
print(tfidf.most_common(50))



Most frequent terms:
[('said', 18494), ('mln', 9160), ('pct', 7551), ('dlrs', 5815), ('year', 5071), ('u', 4607), ('billion', 4321), ('vs', 4317), ('would', 3930), ('lt', 3351), ('last', 2770), ('bank', 2731), ('trade', 2699), ('oil', 2382), ('cts', 2322), ('market', 2320), ('tonnes', 2307), ('net', 2190), ('one', 1998), ('company', 1983), ('new', 1974), ('also', 1826), ('two', 1813), ('prices', 1813), ('government', 1644), ('japan', 1503), ('rate', 1492), ('february', 1488), ('dollar', 1484), ('january', 1455), ('week', 1431), ('may', 1403), ('shares', 1399), ('price', 1328), ('foreign', 1318), ('loss', 1304), ('march', 1278), ('told', 1276), ('exchange', 1272), ('could', 1250), ('production', 1241), ('share', 1228), ('stock', 1226), ('group', 1205), ('april', 1169), ('rates', 1168), ('today', 1163), ('month', 1159), ('shr', 1150), ('rose', 1148)]
Most frequent document terms:
[('said', 4609), ('mln', 2352), ('pct', 2253), ('year', 2200), ('lt', 1987), ('dlrs', 1966), ('u', 1744), ('w

# Basic IR Model
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

## Representation


```
doc -> `[('u', 18), ('said', 16), ('trade', 15), ('japan', 12), ('dlrs', 6), ('exports', 6), ('imports', 5), ('tariffs', 5), ('japanese', 5), ('billion', 5), ('businessmen', 4), ('might', 4), ('electronics', 4), ('taiwan', 4), ('also', 4), ('last', 4), ('year', 4), ('hong', 4), ('kong', 4), ('products', 3)]`
```

In [37]:
def represent(terms):
    counter = Counter()
    for term in terms:
        term = term.lower()
        if term in stopword_list:
            continue
        if not term.isalpha():
            continue
        counter[term] += 1
    return counter
print(represent(documents[0]['content']).most_common(20))

[('u', 18), ('said', 16), ('trade', 15), ('japan', 12), ('dlrs', 6), ('exports', 6), ('imports', 5), ('tariffs', 5), ('japanese', 5), ('billion', 5), ('businessmen', 4), ('might', 4), ('electronics', 4), ('taiwan', 4), ('also', 4), ('last', 4), ('year', 4), ('hong', 4), ('kong', 4), ('products', 3)]


In [36]:
def tfidf(doc_dict, query_dict):
    global num_docs
    global doc_freq
    score = 0
    for term in query_dict:
        if term in doc_dict:
            score += doc_dict[term] * math.log2(num_docs / doc_freq[term])
    return score


def match_docs(docs, query_dict):
    results = []
    for doc in docs:
        doc_dict = represent(doc['content'])
        matched = True
        for term in query_dict:
            if term not in doc_dict:
                matched = False
                break
        if matched:
            results.append(doc)
    return results


def ranked_search(docs, query, score_func):
    query_terms = query.split(" ")
    query_dict = represent(query_terms)
    results = []
    matched_docs = match_docs(docs, query_dict)
    for doc in matched_docs:
        doc_dict = represent(doc['content'])
        score = score_func(doc_dict, query_dict)
        results.append((doc['title'], score))
    
    ranked = sorted(results, key=lambda x: x[1], reverse=True)
    return ranked

    
query = "Japan beef"
results = search(documents, query)
print("Number of matched results: %d" % len(results))
print("\n".join(results[0:10]))

print(
'''
---------------------------------------------------------------
ranked_search
---------------------------------------------------------------
''')
results = ranked_search(documents, query, tfidf)
print("\nNumber of matched results: %d" % len(results))
for title, score in results:
    print("%s \t %f" % (title.lower(), score))
                           

Number of matched results: 21
ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT
JAPAN MINISTRY SAYS OPEN FARM TRADE WOULD HIT U . S .
JAPAN ' S LDP URGES MORE IMPORTS OF 12 FARM ITEMS
U . S . URGES JAPAN TO OPEN FARM MARKET FURTHER U . S .
LYNG OPENS JAPAN TALKS ON FARM TRADE BARRIERS U . S .
U . S . URGES JAPAN TO OPEN FARM MARKET FURTHER U . S .
U . S . MEAT INDUSTRY LAUNCHES CAMPAIGN IN JAPAN
GATT CASE AGAINST JAPAN A MODEL FOR U . S . - LYNG
U . S . TAKES TOUGH STAND ON GATT FARM ISSUES
EC AGREES TRADE DEAL WITH ARGENTINA

---------------------------------------------------------------
ranked_search
---------------------------------------------------------------


Number of matched results: 21
japan beef price support cut will not raise demand 	 168.859461
u . s . meat industry launches campaign in japan 	 145.589919
lyng opens japan talks on farm trade barriers u . s . 	 96.102045
japan ministry says open farm trade would hit u . s . 	 86.204470
japan seen reducing beef , pork 

### Sorting

In [38]:
small_list = [5, 6, 3, 4, 6, 7, 9]
print(sorted(small_list))
print(sorted(small_list, reverse=True))
print(sorted(small_list, key=lambda x: -x))

small_strings = ['beef', 'chiken', 'pork', 'mutton', 'hot']
print(sorted(small_strings))
print(sorted(small_strings, key=lambda x: len(x)))


small_tuples = [('beef', 4), ('chiken', 2), ('pork', 4), ('mutton', 3), ('hot', 7)]
print(sorted(small_tuples))
print(sorted(small_tuples, key=lambda x: x[0]))
print(sorted(small_tuples, key=lambda x: x[1]))
print(sorted(small_tuples, key=lambda x: len(x[0])))


small_strings = ['社會系', '社會工作系', '經濟系', '政治系', '國家發展研究所']
print(sorted(small_strings))
print(sorted(small_strings, key=lambda x: len(x)))


[3, 4, 5, 6, 6, 7, 9]
[9, 7, 6, 6, 5, 4, 3]
[9, 7, 6, 6, 5, 4, 3]
['beef', 'chiken', 'hot', 'mutton', 'pork']
['hot', 'beef', 'pork', 'chiken', 'mutton']
[('beef', 4), ('chiken', 2), ('hot', 7), ('mutton', 3), ('pork', 4)]
[('beef', 4), ('chiken', 2), ('hot', 7), ('mutton', 3), ('pork', 4)]
[('chiken', 2), ('mutton', 3), ('beef', 4), ('pork', 4), ('hot', 7)]
[('hot', 7), ('beef', 4), ('pork', 4), ('chiken', 2), ('mutton', 3)]
['國家發展研究所', '政治系', '社會工作系', '社會系', '經濟系']
['社會系', '經濟系', '政治系', '社會工作系', '國家發展研究所']


## BM25
https://en.wikipedia.org/wiki/Okapi_BM25

In [23]:
avgdl = 0
for doc in documents:
    doc_dict = represent(doc['content'])
    avgdl += sum(doc_dict.values())
avgdl /= num_docs
    

def idf(term):
    global num_docs
    global doc_freq
    return math.log2((num_docs - doc_freq[term] + 0.5) / (doc_freq[term] + 0.5))
                     
                     
def bm25(doc_dict, query_dict):
    global num_docs
    global doc_freq
    global avgdl
    k1 = 1.2
    b = 0.75
    query_len = sum(query_dict.values())
    doc_len = sum(doc_dict.values())
    score = 0
    for term in query_dict:
        if term in doc_dict:
            tf = ((doc_dict[term] / doc_len) * (k1 + 1)) / (
                (doc_dict[term] / doc_len) + k1 *(1 - b + b * (doc_len / avgdl)))
            score += idf(term) * tf
    return score
                                                            
                
results = ranked_search(documents, query, bm25)
print("\nNumber of matched results: %d" % len(results))
for title, score in results:
    print("%s \t %f" % (title.lower(), score))



Number of matched results: 21
japan ' s lipc to buy beef on april 23 	 1.676211
japan ' s lipc to buy beef on april 23 	 1.676211
japan sets 1987 / 88 first half beef import quota 	 1.548682
more pressure urged for asia to take u . s . beef 	 0.618041
japan seen reducing beef , pork intervention prices 	 0.523364
australian beef output seen declining in 1987 	 0.444577
u . s . meat industry launches campaign in japan 	 0.430094
gatt case against japan a model for u . s . - lyng 	 0.426406
japan ' s ldp urges more imports of 12 farm items 	 0.424614
u . s . to ask for share of japan ' s rice market u . s . 	 0.337159
japan beef price support cut will not raise demand 	 0.315739
lyng opens japan talks on farm trade barriers u . s . 	 0.241400
u . s . urges japan to open farm market further u . s . 	 0.233559
u . s . urges japan to open farm market further u . s . 	 0.233559
japan ministry says open farm trade would hit u . s . 	 0.215694
ec agrees trade deal with argentina 	 0.160007
ec


# Inverted Indexing

## Build inverted index

In [110]:
inverted_index = {}

for i in range(len(documents)):
    doc_dict = represent(documents[i]['content'])
    for term in doc_dict:
        if term not in inverted_index:
            inverted_index[term] = []
        inverted_index[term].append(i)
        
    

In [111]:
def match_indexed_docs(docs, inverted_index, query_dict):
    count = Counter()
    for term in query_dict:
        for doc_id in inverted_index[term]:
            count[doc_id] += 1
    results = []
    for doc_id in count:
        if count[doc_id] == len(query_dict):
            results.append(docs[doc_id])
    return results
            
    
def ranked_indexed_search(docs, inverted_index, query, score_func):
    query_terms = query.split(" ")
    query_dict = represent(query_terms)
    results = []
    docs = match_indexed_docs(docs, inverted_index, query_dict)
    for doc in docs:
        doc_dict = represent(doc['content'])
        score = score_func(doc_dict, query_dict)
        if score > 0:
            results.append((doc['title'], score))
    
    ranked = sorted(results, key=lambda x: x[1], reverse=True)
    return ranked


results = ranked_indexed_search(documents, inverted_index, query, bm25)
print("\nNumber of matched results: %d" % len(results))
for title, score in results:
    print("%s %f" % (title, score))



Number of matched results: 21
JAPAN ' S LIPC TO BUY BEEF ON APRIL 23 1.676211
JAPAN ' S LIPC TO BUY BEEF ON APRIL 23 1.676211
JAPAN SETS 1987 / 88 FIRST HALF BEEF IMPORT QUOTA 1.548682
MORE PRESSURE URGED FOR ASIA TO TAKE U . S . BEEF 0.618041
JAPAN SEEN REDUCING BEEF , PORK INTERVENTION PRICES 0.523364
AUSTRALIAN BEEF OUTPUT SEEN DECLINING IN 1987 0.444577
U . S . MEAT INDUSTRY LAUNCHES CAMPAIGN IN JAPAN 0.430094
GATT CASE AGAINST JAPAN A MODEL FOR U . S . - LYNG 0.426406
JAPAN ' S LDP URGES MORE IMPORTS OF 12 FARM ITEMS 0.424614
U . S . TO ASK FOR SHARE OF JAPAN ' S RICE MARKET U . S . 0.337159
JAPAN BEEF PRICE SUPPORT CUT WILL NOT RAISE DEMAND 0.315739
LYNG OPENS JAPAN TALKS ON FARM TRADE BARRIERS U . S . 0.241400
U . S . URGES JAPAN TO OPEN FARM MARKET FURTHER U . S . 0.233559
U . S . URGES JAPAN TO OPEN FARM MARKET FURTHER U . S . 0.233559
JAPAN MINISTRY SAYS OPEN FARM TRADE WOULD HIT U . S . 0.215694
EC AGREES TRADE DEAL WITH ARGENTINA 0.160007
ECONOMIC SPOTLIGHT - U . S . CONGR

In [112]:
query = "Taiwan Japan"
results = ranked_indexed_search(documents, inverted_index, query, bm25)
print("\nNumber of matched results: %d" % len(results))
for title, score in results:
    print("%s %f" % (title, score))



Number of matched results: 41
RISING TAIWAN DOLLAR CAUSES FOREIGN RESERVES LOSS 0.557418
TAIWAN STEEL FIRM SEES LOWER EXPORTS , MORE OUTPUT 0.534024
THAI ZINC EXPORTS FALL IN MARCH 0.447403
LONDON GRAIN FREIGHTS 27 , 000 0.411234
TAIWAN PLANS MISSION TO CLOSE TRADE GAP WITH U . S . 0.361747
TAIWAN FOREIGN EXCHANGE RESERVES HIT NEW HIGH 0.358275
TAIWAN SEES SHARP DECLINE IN SHIPBREAKING 0.304945
YEUTTER PUTS CURRENCY BURDEN ON TAIWAN , KOREA 0.289958
MULFORD SAYS GERMANY , JAPAN SHOULD DO MORE 0.248875
BALDRIGE CONCERNED ABOUT KOREAN / TAIWAN DEFICITS 0.226503
SOUTH AFRICA CORN EXPORTS COULD BE REDUCED - USDA 0.210748
LONDON FREIGHT MARKET FEATURES GRAIN OUT OF U . S . 0.210735
TAIWAN ANNOUNCES NEW ROUND OF IMPORT TARIFF CUTS 0.165164
TAIWAN ANNOUNCES NEW ROUND OF IMPORT TARIFF CUTS 0.165164
BALDRIGE WARNS OF WORLD TRADE WAR DANGER U . S . 0.155060
BALDRIGE WARNS OF WORLD TRADE WAR DANGER U . S . 0.152501
TAIWAN CURBS INFLOWS OF FOREIGN EXCHANGE 0.141761
TAIWAN COMPLAINS ABOUT SIZE OF 

In [113]:
results = ranked_indexed_search(documents, inverted_index, query, tfidf)
print("\nNumber of matched results: %d" % len(results))
for title, score in results:
    print("%s %f" % (title, score))



Number of matched results: 41
ECONOMIC SPOTLIGHT - U . S . DEFICIT WITH 121.213109
TAIWAN SEES SHARP DECLINE IN SHIPBREAKING 64.755111
ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT 63.133312
ECONOMIC SPOTLIGHT - U . S . CONGRESS RAPS JAPAN 45.701311
TAIWAN CURBS INFLOWS OF FOREIGN EXCHANGE 43.998890
NET CHANGE IN EXPORT COMMITMENTS -- USDA 39.863770
TAIWAN ANNOUNCES NEW ROUND OF IMPORT TARIFF CUTS 38.188223
TAIWAN ANNOUNCES NEW ROUND OF IMPORT TARIFF CUTS 38.188223
RISING TAIWAN DOLLAR CAUSES FOREIGN RESERVES LOSS 35.701776
TAIWAN COMPLAINS ABOUT SIZE OF RESERVES 32.377556
TAIWAN FOREIGN EXCHANGE RESERVES HIT NEW HIGH 32.377556
TAIWAN DOLLAR AND RESERVES SEEN RISING MORE SLOWLY 32.377556
JAPANESE TARIFFS SEEN AS WORLDWIDE WARNING 31.566656
ECONOMIC SPOTLIGHT - JAPAN SHIPBUILDERS RECOVERY 25.755989
USDA COMMENTS ON EXPORT SALES REPORT U . S . 24.918215
TAIWAN PLANS MISSION TO CLOSE TRADE GAP WITH U . S . 24.080442
YEUTTER PUTS CURRENCY BURDEN ON TAIWAN , KOREA 24.080442
TAIWAN S